In [51]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataset_path = r'./data/test'

test_master = pd.read_csv(os.path.join(dataset_path, 'Master_Test_Set.csv'), encoding='gbk')
test_userupdateinfo = pd.read_csv(os.path.join(dataset_path, 'Userupdate_Info_Test_Set.csv'), encoding='gbk')
test_loginfo = pd.read_csv(os.path.join(dataset_path, 'LogInfo_Test_Set.csv'), encoding='gbk')

In [52]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
%pprint

Pretty printing has been turned OFF


In [53]:
## 加载微软雅黑中文字体
from matplotlib.font_manager import FontProperties
myfont = FontProperties(fname=r"./SIMHEI.TTF",size=12)

%matplotlib inline
n_null_rate = test_master.isnull().sum().sort_values(ascending=False)/30000
n_null_rate.head(20)
## 去掉缺失比例接近百分之百的字段
test_master.drop(['WeblogInfo_1' ,'WeblogInfo_3'],axis=1,inplace=True)

WeblogInfo_1     0.966267
WeblogInfo_3     0.966267
UserInfo_11      0.635033
UserInfo_12      0.635033
UserInfo_13      0.635033
WeblogInfo_20    0.265900
WeblogInfo_21    0.102000
WeblogInfo_19    0.099033
WeblogInfo_2     0.056867
WeblogInfo_4     0.056700
WeblogInfo_5     0.056700
WeblogInfo_6     0.056700
UserInfo_2       0.008800
WeblogInfo_32    0.008367
WeblogInfo_37    0.008367
WeblogInfo_35    0.008367
WeblogInfo_34    0.008367
WeblogInfo_33    0.008367
WeblogInfo_30    0.008367
WeblogInfo_31    0.008367
dtype: float64

In [54]:
test_master.loc[(test_master.UserInfo_12.isnull() , 'UserInfo_12')] = 2.0
test_master.loc[(test_master.UserInfo_11.isnull() , 'UserInfo_11')] = 2.0
test_master.loc[(test_master.UserInfo_13.isnull() , 'UserInfo_13')] = 2.0
test_master.loc[(test_master.WeblogInfo_20.isnull() , 'WeblogInfo_20')] = u'不详'
test_master.loc[(test_master.WeblogInfo_19.isnull() , 'WeblogInfo_19')] = u'不详'
test_master.loc[(test_master.WeblogInfo_21.isnull() , 'WeblogInfo_21')] = '0'

In [55]:
categoric_cols = ['UserInfo_1' ,'UserInfo_2' ,'UserInfo_3' ,'UserInfo_4' , 'UserInfo_5' ,'UserInfo_6','UserInfo_7','UserInfo_8','UserInfo_9','UserInfo_11','UserInfo_12','UserInfo_13','UserInfo_14','UserInfo_15','UserInfo_16','UserInfo_17','UserInfo_19','UserInfo_20','UserInfo_21','UserInfo_22','UserInfo_23','UserInfo_24','Education_Info1','Education_Info2','Education_Info3','Education_Info4','Education_Info5','Education_Info6','Education_Info7','Education_Info8','WeblogInfo_19','WeblogInfo_20','WeblogInfo_21','SocialNetwork_1','SocialNetwork_2','SocialNetwork_7','SocialNetwork_12']
# for col in categoric_cols:
#     mode_cols = test_master[col].mode()[0]
#     test_master.loc[(test_master[col].isnull() , col)] = mode_cols 

## 用均值填充缺失值 
numeric_cols = []
for col in test_master.columns:
    if col not in categoric_cols and col !=u'Idx' and  col !='ListingInfo':
        mean_cols = test_master[col].mean()
        test_master.loc[(test_master[col].isnull() , col)] = mean_cols


In [56]:
import re
## 去掉空格
test_master['UserInfo_9'] = test_master['UserInfo_9'].apply(lambda x: x.strip())
## 去掉大小写
test_userupdateinfo['UserupdateInfo1'] =test_userupdateinfo['UserupdateInfo1'].apply(lambda x:x.lower())
## 将UserInfo_8中城市名归一化
def encodingstr(s):
    regex = re.compile(r'.+市')
    if regex.search(s):
        s = s[:-1]
        return s
    else:
        return s
test_master['UserInfo_8'] =test_master['UserInfo_8'].apply(lambda x: encodingstr(x))
test_userupdateinfo.to_csv('./test_userupdateinfo.csv',index=False,encoding='utf-8')

In [57]:
## UserInfo_2处理
dummies_UserInfo_2 = pd.get_dummies(test_master['UserInfo_2'], prefix='UserInfo_2')
dummies_UserInfo_2.head()
dummies_UserInfo_2.shape

,UserInfo_2_七台河,UserInfo_2_三明,UserInfo_2_三门峡,UserInfo_2_上海,UserInfo_2_上饶,UserInfo_2_东莞,UserInfo_2_东营,UserInfo_2_中山,UserInfo_2_临夏回族自治州,UserInfo_2_临汾,...,UserInfo_2_黄冈,UserInfo_2_黄南藏族自治州,UserInfo_2_黄山,UserInfo_2_黄石,UserInfo_2_黑河,UserInfo_2_黔东南,UserInfo_2_黔南,UserInfo_2_黔西南,UserInfo_2_齐齐哈尔,UserInfo_2_龙岩
0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


(30000, 331)

In [58]:

## 剔除标准差几乎为零的特征项
feature_std = test_master.std().sort_values(ascending=True)
feature_std.head(20)
test_master.drop([], axis=1, inplace=True)
test_master['Idx'] = test_master['Idx'].astype(np.int32)

for i in range(25):
    name = 'UserInfo_' + str(i)
    try:
        print(test_master[name].head())
    except:
        pass

test_master['UserInfo_8'].head(20)

TypeError: could not convert string to float: '中山'

In [60]:
## 借款日期离散化
# 把月、日、单独拎出来，放到3列中
test_master['month'] = pd.DatetimeIndex(test_master['ListingInfo']).month
test_master['day']  = pd.DatetimeIndex(test_master['ListingInfo']).day
test_master.drop(['ListingInfo'],axis=1,inplace=True)
test_master.to_csv('./test_master.csv',index=False,encoding='utf-8')

C:\Users\Redamancy\AppData\Local\Temp\ipykernel_18912\1799553785.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_master['month'] = pd.DatetimeIndex(test_master['ListingInfo']).month
C:\Users\Redamancy\AppData\Local\Temp\ipykernel_18912\1799553785.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_master['day']  = pd.DatetimeIndex(test_master['ListingInfo']).day


In [61]:
test_master

,Idx,UserInfo_1,UserInfo_2,UserInfo_3,UserInfo_4,WeblogInfo_2,WeblogInfo_4,WeblogInfo_5,WeblogInfo_6,WeblogInfo_7,...,SocialNetwork_10,SocialNetwork_11,SocialNetwork_12,SocialNetwork_13,SocialNetwork_14,SocialNetwork_15,SocialNetwork_16,SocialNetwork_17,month,day
0,10,5.0,中山,5.0,中山,0.0,3.027033,1.832326,2.944238,0.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,11,1
1,100,1.0,商丘,3.0,郑州,0.0,3.000000,2.000000,3.000000,0.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,11,9
2,1000,1.0,黔东南,3.0,黔东南,0.0,5.000000,2.000000,5.000000,0.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,11,19
3,10000,6.0,上海,6.0,上海,0.0,1.000000,1.000000,1.000000,4.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,2,23
4,10009,5.0,上海,1.0,上海,1.0,4.000000,3.000000,4.000000,17.0,...,11.0,-1.0,0,0.0,0.0,0.0,0.0,1.0,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,998,5.0,北京,5.0,包头,0.0,1.000000,1.000000,1.000000,0.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,11,21
29996,9985,1.0,西安,5.0,西安,0.0,2.000000,1.000000,2.000000,9.0,...,-1.0,-1.0,-1,0.0,0.0,0.0,0.0,0.0,2,24
29997,9988,4.0,百色,1.0,百色,0.0,3.000000,1.000000,3.000000,14.0,...,-1.0,-1.0,-1,1.0,0.0,0.0,0.0,0.0,2,28
29998,9996,3.0,常州,5.0,镇江,1.0,12.000000,2.000000,12.000000,27.0,...,-1.0,-1.0,-1,1.0,0.0,0.0,0.0,0.0,3,6


In [62]:
from collections import defaultdict
import datetime as dt

##  userupdateinfo表
userupdate_info_number = defaultdict(list)  ### 用户信息更新的次数
userupdate_info_category = defaultdict(set)  ###用户信息更新的种类数
userupdate_info_times = defaultdict(list)  ### 用户分几次更新了
userupdate_info_date = defaultdict(list)  #### 用户借款成交与信息更新时间跨度
with open('./test_userupdateinfo.csv', 'r') as f:
    f.readline()
    for line in f.readlines():
        cols = line.strip().split(",")  ### cols 是list结果
        userupdate_info_date[cols[0]].append(cols[1])
        userupdate_info_number[cols[0]].append(cols[2])
        userupdate_info_category[cols[0]].add(cols[2])
        userupdate_info_times[cols[0]].append(cols[3])
    print(u'提取信息完成')


'Idx,ListingInfo1,UserupdateInfo1,UserupdateInfo2\n'

提取信息完成


In [63]:

userupdate_info_number_ = defaultdict(int)  ### 用户信息更新的次数
userupdate_info_category_ = defaultdict(int)  ### 用户信息更新的种类数
userupdate_info_times_ = defaultdict(int)  ### 用户分几次更新了
userupdate_info_date_ = defaultdict(int)  #### 用户借款成交与信息更新时间跨度
for key in userupdate_info_date.keys():
    userupdate_info_times_[key] = len(set(userupdate_info_times[key]))
    delta_date = dt.datetime.strptime(userupdate_info_date[key][0], '%Y/%m/%d') - dt.datetime.strptime(
        list(set(userupdate_info_times[key]))[0], '%Y/%m/%d')
    userupdate_info_date_[key] = abs(delta_date.days)
    userupdate_info_number_[key] = len(userupdate_info_number[key])
    userupdate_info_category_[key] = len(userupdate_info_category[key])

print('信息处理完成')


信息处理完成


In [64]:
## 建立一个DataFrame
Idx_ = list(userupdate_info_date_.keys())  #### list
numbers_ = list(userupdate_info_number_.values())
categorys_ = list(userupdate_info_category_.values())
times_ = list(userupdate_info_times_.values())
dates_ = list(userupdate_info_date_.values())
userupdate_df = pd.DataFrame(
    {'Idx': Idx_, 'numbers': numbers_, 'categorys': categorys_, 'times': times_, 'dates': dates_})
userupdate_df.head()
userupdate_df.to_csv('./userupdate_df.csv', index=False, encoding='utf-8')

,Idx,numbers,categorys,times,dates
0,10,12,11,2,35
1,100,12,11,2,10
2,1000,13,11,2,7
3,10000,10,9,1,3
4,10009,21,14,3,5


In [66]:
test_master = pd.read_csv('./test_master.csv', encoding='utf-8')
test_userupdateinfo = pd.read_csv('./userupdate_df.csv', encoding='utf-8')
#test_loginfo = pd.read_csv('./loginfo_df.csv',encoding='utf-8')
# test_all = pd.merge(test_master, test_userupdateinfo, how='left', on='Idx')
#test_all = pd.merge(test_all, test_loginfo, how='left', on='Idx')
test_all=test_master
test_all.isnull().sum().sort_values(ascending=False).head(10)
test_all['Idx'] = test_all['Idx'].astype(np.int64)
test_all = pd.get_dummies(test_all)
test_all.head()
test_all.to_csv('./test_all.csv',encoding='utf-8',index=False)

UserInfo_2                    264
UserInfo_4                    239
UserInfo_3                      4
UserInfo_1                      4
Idx                             0
ThirdParty_Info_Period4_12      0
ThirdParty_Info_Period4_16      0
ThirdParty_Info_Period4_15      0
ThirdParty_Info_Period4_14      0
ThirdParty_Info_Period4_13      0
dtype: int64

,Idx,UserInfo_1,UserInfo_3,WeblogInfo_2,WeblogInfo_4,WeblogInfo_5,WeblogInfo_6,WeblogInfo_7,WeblogInfo_8,WeblogInfo_9,...,WeblogInfo_20_I7,WeblogInfo_20_I8,WeblogInfo_20_O,WeblogInfo_20_U,WeblogInfo_20_不详,WeblogInfo_21_0,WeblogInfo_21_A,WeblogInfo_21_B,WeblogInfo_21_C,WeblogInfo_21_D
0,10,5.0,5.0,0.0,3.027033,1.832326,2.944238,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
1,100,1.0,3.0,0.0,3.000000,2.000000,3.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
2,1000,1.0,3.0,0.0,5.000000,2.000000,5.000000,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
3,10000,6.0,6.0,0.0,1.000000,1.000000,1.000000,4.0,0.0,0.0,...,False,False,False,False,True,False,False,False,False,True
4,10009,5.0,1.0,1.0,4.000000,3.000000,4.000000,17.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
